In [21]:
import numpy as np

def estimator(num_days_all, num_days_section, num_days_burnin, atol = 0.01):
    
    try:
        _cache, _delta = 0, np.inf
        while (_delta > atol):
            _tmp = (num_days_all + _cache * num_days_burnin) / num_days_section
            print(_tmp)
            
            _delta = _tmp - _cache
            _cache = _tmp

        return True
    
    except Exception as _e:
        print(_e)
        return False

In [22]:
estimator(243 * 2.75, 48, 10)

13.921875
16.822265625
17.426513671875
17.552398681640625
17.578624725341797
17.58408848444621


True

# PO Data

In [ ]:
import pandas as pd

from utils.tcm_gen import TCMGen
from data_core import *

In [ ]:
HOME_REMOTE = "/home/hopec"
paths = {
    "config": {
        "local": "data/po/config/optCfg.white_PO_prod.HKGUniv.EqTOP500.csv",
        "remote": "po_root/config/HKGUniv.EqTOP500/optCfg.white_PO_prod.HKGUniv.EqTOP500.csv"
    },
    "tcm": {
        "local": "/home/hope/data/rch/eq/tcm/tcmStats/HKGUniv.EqTOP500/DAILY/TCMStats.HKGUniv.EqTOP500.{}.csv.gz",
        "remote": "po_root/data/eq/tcm/tcmStats/HKGUniv.EqTOP500/DAILY/TCMStats.HKGUniv.EqTOP500.{}.csv.gz"
    },
    "adv63": {
        "local": "/home/hope/simDriver/running_pool/analysis/white_po/data/day_vol_adv63/{}.parquet",
        "remote": "po_root/data/local/volume_prediction/adv63/1.0.0/{}.csv.gz"
    }
}

## tcm

In [43]:
ra = constructor_set_region_asset("hkg", "eq")
date_list = ra.dates.get_trading_dates(20220901, 20221024, fmt = "int")
tcm_gen = TCMGen(["HKGUniv.EqTOP500",
                  "2022-09-01",
                  "2022-10-24",
                  "/home/hope/mta_nas/rch"])
for date in date_list:
    _str_date = "-".join([str(date)[:4], str(date)[4:6], str(date)[6:]])
    
    _df = tcm_gen.tcm_gen(date)
    _df.to_csv(paths["tcm"]["local"].format(_str_date),
               index = False)

TypeError: 'NoneType' object does not support item assignment

## adv63

In [46]:
import time

In [48]:
region_asset = constructor_set_region_asset("hkg", "eq")

_n_backward = 63
PATH_OUTPUT = '/data/home/hope/simDriver/running_pool/analysis/white_po/data/day_vol_adv%i/'%_n_backward
if not os.path.exists(PATH_OUTPUT):
    os.mkdir(PATH_OUTPUT)
else:
    print('WARNING Overwriting files in %s'%PATH_OUTPUT)
dateList = region_asset.dates.get_trading_dates(20220901, 20221021, fmt = 'int')

for dd in dateList:
    
    time1 = time.time()
    
    stock_member = region_asset.univs.get_all_jkeys("HKGUniv.EqTOP500", dd, dd)
    config_params = GenericTabularData.ConfigParams()
    config_params.version = "1.0.3"
    config_params.freq = "1MIN"
    day_vol = GenericTabularData(
        region = 'hkg',
        asset = 'eq',
        dataset = "md_bar1m",
        univ = stock_member,
        start_date = region_asset.dates.shift_trading_dates(dd, -_n_backward),
        end_date = region_asset.dates.shift_trading_dates(dd, -1),
        config_params = config_params
    ).as_data_frame()
    
    day_vol = day_vol.groupby(['jkey', 'hhmmss_nano'])[['amount']].mean().reset_index()
    day_vol['yyyymmdd'] = dd
    
    day_vol.to_parquet(os.path.join(PATH_OUTPUT, '%i.parquet'%dd), index = False)
    
    print(dd, 'done; time = %.2f sec'%(time.time() - time1))
    
    
# _n_backward = 63
# PATH_OUTPUT = '/data/home/hope/simDriver/running_pool/analysis/white_po/data/day_vol_adv%i/'%_n_backward
# print('A total of %i files found'%(len(os.listdir(PATH_OUTPUT))))

# for file in os.listdir(PATH_OUTPUT):
    
#     dd = int(file.split('.')[0])
    
#     write_file_to_jdfs(os.path.join(PATH_OUTPUT, file),
#                        '/strat_res/strat_res_input_HKG/volume_prediction/adv63/1.0.0/%i.parquet'%dd,
#                        'strat_res', 'strat_res_input_HKG')

WARNING Overwriting files in /data/home/hope/simDriver/running_pool/analysis/white_po/data/day_vol_adv63/
20220901 done; time = 37.55 sec
20220902 done; time = 35.17 sec
20220905 done; time = 36.04 sec
20220906 done; time = 34.87 sec
20220907 done; time = 35.37 sec
20220908 done; time = 35.59 sec
20220909 done; time = 34.90 sec
20220913 done; time = 36.13 sec
20220914 done; time = 36.57 sec
20220915 done; time = 38.62 sec
20220916 done; time = 34.57 sec
20220919 done; time = 36.14 sec
20220920 done; time = 35.58 sec
20220921 done; time = 36.30 sec
20220922 done; time = 35.57 sec
20220923 done; time = 37.26 sec
20220926 done; time = 35.25 sec
20220927 done; time = 38.59 sec
20220928 done; time = 36.66 sec
20220929 done; time = 35.20 sec
20220930 done; time = 34.70 sec
20221003 done; time = 35.28 sec
20221005 done; time = 34.21 sec
20221006 done; time = 36.10 sec
20221007 done; time = 35.98 sec
20221010 done; time = 37.32 sec
20221011 done; time = 34.56 sec
20221012 done; time = 35.17 se

In [49]:
HOME_REMOTE = "/home/hopec/"
paths = {
    "config": {
        "local": "data/po/config/optCfg.white_PO_prod.HKGUniv.EqTOP500.csv",
        "remote": "po_root/config/HKGUniv.EqTOP500/optCfg.white_PO_prod.HKGUniv.EqTOP500.csv"
    },
    "tcm": {
        "local": "/home/hope/data/rch/eq/tcm/tcmStats/HKGUniv.EqTOP500/DAILY/TCMStats.HKGUniv.EqTOP500.{}.csv.gz",
        "remote": "po_root/data/eq/tcm/tcmStats/HKGUniv.EqTOP500/DAILY/TCMStats.HKGUniv.EqTOP500.{}.csv.gz"
    },
    "adv63": {
        "local": "/home/hope/simDriver/running_pool/analysis/white_po/data/day_vol_adv63/{}.parquet",
        "remote": "po_root/data/local/volume_prediction/adv63/1.0.0/{}.csv.gz"
    }
}

In [51]:
import pandas as pd

In [52]:
ra = constructor_set_region_asset("hkg", "eq")
date_list = ra.dates.get_trading_dates(20220906, 20221021, fmt = "int")

for date in date_list:
    _df = pd.read_parquet(paths["adv63"]["local"].format(date))
    _df.to_csv("data/tmp/{}.csv.gz".format(date), index = False)
    
    os.system("scp data/tmp/{}.csv.gz hopec@10.8.64.176:{}"\
              .format(date,
                      (HOME_REMOTE + paths["adv63"]["remote"]).format(date)))
    os.remove("data/tmp/{}.csv.gz".format(date))

## msci